# 第9章: ベクトル空間法(I)
enwiki-20150112-400-r10-105752.txt.bz2は、2015年1月12日時点の英語のwikipedia記事のうち、約400語以上で構成される記事の中からランダムにサンプリングした105752記事のテキストをbzip2形式で圧縮したものである。このテキストをコーパスとして単語の意味を表すベクトル(分散表現)を学習したい。第9章の前半では、コーパスから作成した単語文脈共起行列に主成分分析を適用し、単語ベクトルを学習する過程を、いくつかの処理に分けて実装する。第9章の後半では、学習で得られた単語ベクトル(300語)を用い、単語の類似度計算やアナロジー(類推)を行う。

In [2]:
import re
import sys
from random import randint
from collections import defaultdict
from tqdm import tqdm
import json
import pickle
from scipy import sparse
import numpy as np
import csv


### 80. コーパスの整形
文を単語列に変換する最も単純な方法は、空白文字で単語を区切ることである。ただこの方法では文末のピリオドや括弧などの記号が単語に含まれてしまう。そこでコーパスの各行のテキストを空白文字でトークンのリストに分割したのち、各トークンに以下の処理を施し、単語から記号を除去せよ。

* トークンの先頭と末尾に出現する次の文字を削除: `.,!?;:()[]'"`
* 空文字となったトークンは削除  
以上の処理を適用した後、トークンをスペースで連結してファイルに保存せよ。

In [2]:
rm_punctuation = lambda x:re.sub('[.,!?;:()\[\]\'"]','',x)
Sentence = open('./enwiki-20150112-400-r10-105752.txt','r')
ret_file = open('./formated_corpus.txt', 'w')
for line in Sentence:
    temp_line = line.split(' ')
    temp_line = list(map(rm_punctuation, temp_line))
    temp_line = list(filter(lambda x:x!='', temp_line))
    temp_line = " ".join(temp_line)
    ret_file.writelines(temp_line)
ret_file.close()

### 81.複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [3]:
countries = open('./Country.txt','r')
#国複合語を抽出
complex_words = {}
for country in countries:
    if len(country.split()) > 1:
        complex_words[country.split()[0]] = country.rstrip()

In [4]:
file = open('./formated_corpus.txt','r')
ret_file = open('./complex_words_conbine.txt','w')

skip_count = 0
for line in file:
    temp_line = []
    for i, token in enumerate(line.split(' ')):
        
        if skip_count > 0:
            skip_count -=1
            continue
        
        if token in complex_words:
            complex_word_List = complex_words[token].split()
            flag = True
            for j in range(len(complex_word_List)):
                essense = complex_word_List[j]
                try:
                    temp = line.split(' ')[i+j]
                except:
                    temp = "XXX"
                if temp != essense:
                    flag = False
                    
            if flag:
                temp_line.append("_".join(complex_word_List))
                skip_count = len(complex_word_List)
        else:
            temp_line.append(token)
    ret_file.write(" ".join(temp_line))
ret_file.close()
                           
                

### 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語*t*に関して単語*t*と文脈語*c*のペアをタブ区切りですべて書き出せ。ただし文脈語の定義は以下の通り。
* ある単語*t*の前後*d*単語を文脈語cとして出力する(ただし、文脈語に単語*t*そのものは含まない).
* 単語*t* を選ぶ度に、文脈幅*d*は```{1,2,3,4,5}```の範囲でランダムに決める

In [5]:
short_src = open('./short.txt','r')
file = open('./complex_words_conbine.txt','r')
ret_file = open('./token_context.txt','w')
for line in file:
    iter_tokens  = line.split()
    if len(iter_tokens) == 1:
        continue
    
    for i in range(1,len(iter_tokens)):
        if i < 5:
            d = randint(1,i)
        elif len(iter_tokens)-i<5:
            d = randint(1,len(iter_tokens)-i)
        else:
            d = randint(1,5)
        ret_file.write(iter_tokens[i]+"\t"+"\t".join(iter_tokens[i-d:i]+iter_tokens[i+1:i+d+1])+'\n')
    
ret_file.close()

### 83. 単語/ 文脈の頻度の計測
82の出力を利用し、以下の出現分布、および定数を求めよ
* $f(t,c)$: 単語*t*と文脈語*c*の共起回数
* $f(t,*)$: 単語*t*の出現回数
* $f(*,c)$: 文脈語cの出現回数
* $N$:単語と文脈語のペアの総出現回数

```bash:
split -l 10000000 ./token_context.txt ./token_context_split
```
を実行しファイルを分割しておく。

In [6]:
for i in range(12):
    file = open('resource/token_context_split'+str(i),'r')
    temp_tc = defaultdict(int)
    temp_t = defaultdict(int)
    temp_c = defaultdict(int)
    
    for line in tqdm(file):
        t = line.split()[0]
        context_words = line.split()[1:]
        temp_t[t] += 1
        for c in context_words:
            temp_c[c] += 1
            temp_tc[(t,c)] +=1
    with open('resource/tc_'+str(i)+'.pkl', 'wb') as tc_file:
        pickle.dump(temp_tc,tc_file)
    
    
    with open('resource/t_'+str(i)+'.pkl','wb') as t_file:
        pickle.dump(temp_t,t_file)
        
    with open('resource/c_'+str(i)+'.pkl','wb') as c_file:
        pickle.dump(temp_c,c_file)
    
    file.close()
    

10000000it [01:15, 131918.79it/s]
10000000it [01:14, 133976.65it/s]
10000000it [01:14, 134956.80it/s]
10000000it [01:14, 135128.10it/s]
10000000it [01:15, 131606.03it/s]
10000000it [01:14, 133634.51it/s]
10000000it [01:13, 135903.66it/s]
10000000it [01:14, 134088.84it/s]
10000000it [01:14, 134552.97it/s]
10000000it [01:13, 135786.87it/s]
10000000it [01:14, 134776.51it/s]
7013682it [00:51, 135126.05it/s]


In [7]:

main_tc = defaultdict(int)
main_t = defaultdict(int)
main_c = defaultdict(int)

for i in tqdm(range(12)):
    with open('./resource/tc_{}.pkl'.format(str(i)),'rb') as tc:
        temp_tc = pickle.load(tc)
        for key, item in temp_tc.items():
            main_tc[",".join(key)] += item
          
    with open('./resource/t_{}.pkl'.format(str(i)),'rb') as t:
        temp_t = pickle.load(t)
        for key,item in temp_t.items():
            main_t[key] +=item  
            
    with open('./resource/c_{}.pkl'.format(str(i)),'rb') as c:
        temp_c = pickle.load(c)
        for key,item in temp_c.items():
            main_c[key] +=item

with open('resource/main_tc.json','w') as tc_file:
    json.dump(main_tc, tc_file)

with open('resource/main_t.json','w') as t_file:
    json.dump(main_t, t_file)
    
with open('resource/main_c.json','w') as c_file:
    json.dump(main_c, c_file)
        
        


100%|██████████| 12/12 [15:01<00:00, 75.14s/it]


In [11]:
csv_List = []
with open('resource/main_tc.csv','w') as tc_csv:
    writer = csv.writer(tc_csv)
    for key,item in main_tc.items():
        temp = key.split(',')
        temp.append(item)
        csv_List.append(temp)
        
    writer.writerows(csv_List)

### 84. 単語文脈行列の作成
83の出力を利用し、単語文脈行列$X$を作成せよ。ただし、行列$X$の各要素$X_{tc}$は次のようにする。
* $f(t,c) \geq 10 \Rightarrow X_{tc} = PPMI(t,c) = max\{\log{\frac{N\times f(t,c)}{f(t,*) \times f(*, c)}}, 0\}$
* $f(t,c) < 10 \Rightarrow X_{tc} = 0$

ここで$PPMI(t,c)$はPositive Pointwise Mutual Infomation(正の相互情報量)と呼ばれる統計量である。なお、行列$X$の行数、列数は数百万オーダーになり、行列すべての要素を主記憶上に乗せることは無理なので注意すること。幸い、行列Xのほとんどの要素は0になるので、非0の要素だけ書き出す。

```bash
    wc -l ./resource/main_tc.csv
```
でNを調べる。

In [8]:
with open('./resource/main_t.json','r') as t_file:
    dict_t = json.load(t_file)
    dict_t_keys = list(dict_t.keys())
with open('./resource/main_c.json','r') as c_file:
    dict_c = json.load(c_file)
    dict_c_keys = list(dict_c.keys())


OSError: [Errno 22] Invalid argument

In [10]:
rows = []
cols = []
data =[]
N =  123075730
with open('./resource/main_tc.csv','r') as tc_file:
    reader = csv.reader(tc_file)
    for i,row in tqdm(enumerate(reader)):
        if int(row[2]) >= 10:
            temp = np.log((N*int(row[2]))/(dict_t[row[0]]*dict_c[row[1]]))
            if temp != 0:
                rows.append(dict_t_keys.index(row[0]))
                cols.append(dict_c_keys.index(row[1]))
                data.append(temp)

123075730it [58:47, 34891.63it/s]


In [12]:
with open('./resource/rows.pkl','wb') as row_file:
    pickle.dump(rows,row_file)
    
with open('./resource/cols.pkl','wb') as col_file:
    pickle.dump(cols,col_file)
    
with open('./resource/data.pkl','wb') as data_file:
    pickle.dump(data,data_file)

### 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して、主成分分析を適用し、単語の意味ベクトルを300次元に圧縮せよ

In [13]:
with open('./resource/rows.pkl','rb') as row_file:
    rows = pickle.load(row_file)
    
with open('./resource/cols.pkl','rb') as col_file:
    cols = pickle.load(col_file)
    
with open('./resource/data.pkl','rb') as data_file:
    data = pickle.load(data_file)

In [14]:
length_row = len(dict_t)
length_col = len(dict_c)

X = sparse.coo_matrix((data, (rows, cols)),shape=(length_row,length_col))

In [15]:
X.shape

(1715039, 1739536)

In [16]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300)
X300dim = svd.fit_transform(X)

In [ ]:
np.save('./resource/X300dim.npy',X300dim)

### 86.単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み、"United States"のベクトルを表示せよ。ただし、"United States"は内部的には"United_States"と表現されていることに注意せよ。

In [20]:
UnitedState_index = dict_t_keys.index('United_States')
#UnitedState_index = 273
X300dim[UnitedState_index]

array([-1.27970484e+01,  5.77445233e+00,  4.40833334e+00,  1.02726818e+01,
       -7.88510507e+00, -5.18350564e-01,  6.49889441e+00, -4.15398408e+00,
       -7.68903868e+00,  1.13597390e-01,  2.03063086e+00, -5.21957474e+00,
       -2.96056246e+00, -3.18377984e+00, -7.61399844e-01, -3.29820382e+00,
       -1.58915078e+00,  1.01529873e-01,  5.35748852e-01, -7.30815457e-01,
        2.33993160e-01,  1.19994821e+00,  4.86317120e+00,  5.58557639e-01,
       -1.31066916e+00,  4.19081722e+00,  3.33579726e+00,  4.30344407e-01,
       -3.56524589e+00, -1.76653084e+00,  9.26803288e-01, -5.46630096e+00,
        1.39431944e+00,  8.32795348e-01, -3.97909784e+00,  6.18217288e+00,
        3.08817651e-01,  7.97488301e-01, -2.25564231e-01,  9.45901213e-02,
       -3.71322125e+00, -5.85453631e+00,  1.18087828e+00, -4.97556555e+00,
        5.43871182e+00,  2.23812650e+00, -1.96665694e-01, -1.08507876e+00,
       -8.17299060e-01,  3.61558335e+00, -8.54163162e-01, -4.62555251e+00,
        2.25217188e-01,  

### 87. 単語の類似度
85で得た単語ベクトルを読み込み、"United States"と"U.S."のコサイン類似度を計算せよ。ただし、"U.S."は内部的には"U.S"と表現されていることに注意せよ

In [21]:
US_index = dict_t_keys.index("US")
#US_index = 1190
def calc_cossim(vec1, vec2):
    inner_product = np.dot(vec1,vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return inner_product/(norm1*norm2)

US_vector = X300dim[US_index]
United_States_vector = X300dim[UnitedState_index]

cos_similarity = calc_cossim(US_vector,United_States_vector)
cos_similarity

0.7112571401815301

### 88.類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み、"England"とコサイン類似度が高い10語と、その類似度を出力せよ。

In [22]:
England_vec = X300dim[dict_t_keys.index("England")]

result_List = []
for i, key in enumerate(dict_t_keys):
    if len(result_List) < 11:
        result_List.append([key, calc_cossim(England_vec, X300dim[i])])
        result_List = sorted(result_List,key=lambda x:x[1],reverse=True)
    else:
        temp_cossim = calc_cossim(England_vec, X300dim[i])
        if result_List[-1][1] < temp_cossim:
            result_List[-1] = [key, calc_cossim(England_vec, X300dim[i])]
            result_List = sorted(result_List,key=lambda x:x[1], reverse=True)

result_List[1:]

/Users/home/.pyenv/versions/3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


[['Scotland', 0.7605852369214897],
 ['Ireland', 0.6730711277946991],
 ['Wales', 0.6674960683642336],
 ['Australia', 0.6318441208524082],
 ['Britain', 0.593913842860392],
 ['France', 0.5666111035785446],
 ['Zealand', 0.5435728637656047],
 ['Yorkshire', 0.5377893857087538],
 ['London', 0.5254583203765075],
 ['Cornwall', 0.5201827203472809]]

### 89.加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み、vec("Spain") - vec("Madrid") + vec("Athens")を計算し、そのベクトル類似度の高い10語とその類似度を出力せよ

In [23]:
vector = X300dim[dict_t_keys.index("Spain")] - X300dim[dict_t_keys.index("Madrid")] + X300dim[dict_t_keys.index("Athens")]
result_List = []

for i, key in enumerate(dict_t_keys):
    if len(result_List) < 20:
        result_List.append([key, calc_cossim(vector, X300dim[i])])
        result_List = sorted(result_List,key=lambda x:x[1],reverse=True)
    else:
        temp_cossim = calc_cossim(England_vec, X300dim[i])
        if result_List[-1][1] < temp_cossim:
            result_List[-1] = [key, calc_cossim(vector, X300dim[i])]
            result_List = sorted(result_List,key=lambda x:x[1], reverse=True)

result_List[1:]

/Users/home/.pyenv/versions/3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


[['medallist', 0.2978921165189105],
 ['Muhlenberg', 0.27040080770255],
 ['Decatur', 0.2666470083779609],
 ['Paralympic', 0.2600490699019406],
 ['Fairfield', 0.24787179297623158],
 ['Nagano', 0.2379682882908366],
 ['Lamar', 0.23294836506941743],
 ['Macomb', 0.23289291965960707],
 ['Placer', 0.23241486725196886],
 ['Greenville', 0.22719739939274197],
 ['Riverside', 0.22507256052446664],
 ['Gwinnett', 0.2211096478067799],
 ['Diocesan', 0.22101289520528253],
 ['Helsinki', 0.22088225282195154],
 ['medallists', 0.21793807358108577],
 ['Goodwill', 0.21788502095898132],
 ['Boulder', 0.2145588101433165],
 ['Northeastern', 0.2130245907592803],
 ['Preziosa', 0.16575165265821334]]